# Function Calling

Function calling is a powerful feature of OpenAI `gpt-4` and other models. It essentially is a **Function Picker and Parameter Provider**. It can help you choose which function, if any, to invoke with which parameters, under provided context and instruction. 

LionAGI allows simple usage of function callings in the `Session` object. 

In [2]:
tool_schema = {
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Perform multiplication on two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "number1": {
                    "type": "number",
                    "description": "a number to multiply, e.g. 5.34",
                },
                "number2": {
                    "type": "number",
                    "description": "a number to multiply, e.g. 17",
                },
            },
            # specify which parameters are required for the model to respond 
            # when function calling
            "required": ["number1", "number2"],
        },
    }
}

def multiply(number1, number2):
    return number1*number2

In [3]:
from lionagi.core.schema import Tool

# created a tool object
tools = Tool(func=multiply, schema_=tool_schema)

In [4]:
# add some messages for our session
system = """
    you are asked to perform as a function picker and parameter provider
"""

task = """
    Think step by step, understand the following basic math question and 
    provide parameters for function calling.
"""

# json format for the task
json_format = {"number1": "x", "number2": "y"}
instruct1 = {"Task": task, "json_format": json_format}

In [6]:
from lionagi import Session

# now we provide the speific problems as contexts 
question1 = """
    There are [basketball, football, backpack, water bottle, strawberry, 
    tennis ball, rockets]. each comes in four different colors, what is 
    the number of unique kinds of ball?
"""

question2 = """
    There are three fruits in total, each with 2 different colors, how 
    many unique kinds of fruits are there?
"""

session = Session(system=system, tools=tools)

response = await session.chat(
    instruct1, 
    context={"Question1": question1, "question2": question2}, 
    tools=True, 
    response_format={'type':"json_object"}
)

for function call to work, you either need to enforce 

- `response_format={'type':"json_object"}`, or `tool_choice = 'auto'` 

- if you use 'auto', you let model choose what to do, 

- if you want to force a function call, use `tool_choice={"type": "function", "function": {"name": "my_function"}}`

In [7]:
# responses from action requests are dictionary
import lionagi as li

print(li.to_readable_dict(response))

{'action_request': [{'action': 'action_multiply', 'arguments': '{"number1": 2, "number2": 4}'}, {'action': 'action_multiply', 'arguments': '{"number1": 3, "number2": 2}'}]}


In [8]:
li.lcall(response['action_request'], lambda x: print(x));

{'action': 'action_multiply', 'arguments': '{"number1": 2, "number2": 4}'}
{'action': 'action_multiply', 'arguments': '{"number1": 3, "number2": 2}'}


In [9]:
# messages is a pandas dataframe
session.messages

,node_id,role,name,timestamp,content
0,7bd2b4351478a552933de0043f246780,system,system,2024-01-18 12:14:11.789018,"{""system_info"": ""you are asked to perform as a..."
1,3a9a162adc8495e7a2ae5c81fb284048,user,user,2024-01-18 12:14:11.789648,"{""instruction"": {""Task"": ""Think step by step, ..."
2,77bea66b96f6beb00bb02836c5f90cfa,assistant,action_request,2024-01-18 12:14:20.591398,"{""action_request"": [{""action"": ""action_multiply"",..."
3,fe7aa5a565c0244ce99c56b12aba8529,assistant,action_response,2024-01-18 12:14:20.592549,"{""action_response"": {""function"": ""multiply"", ""..."
4,3ac75169c83d3e442dcc9418968ced99,assistant,action_response,2024-01-18 12:14:20.593214,"{""action_response"": {""function"": ""multiply"", ""..."


In [10]:
# since the response is an output from action, you need to use the key `action_response`
# now let us check the function responses
df = session.default_branch.action_response

for content in df['content']:
    dict_ = li.to_dict(content)
    print(dict_)

{'function': 'multiply', 'arguments': {'number1': 2, 'number2': 4}, 'output': 8}
{'function': 'multiply', 'arguments': {'number1': 3, 'number2': 2}, 'output': 6}
